In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 486 kB 55.5 MB/s 
     |████████████████████████████████| 13.4 MB 49.2 MB/s 
     |████████████████████████████████| 880 kB 39.1 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=795b7038861e10cff7d8246dcd1725a1671a5cf7d832376322225e19d4e178f6
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=cc649d07c425b7668ac26a621338ebe47070e46ebf76940b15b1466429793171
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic sacremoses


In [2]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging


In [3]:
batch_size = 16
max_len = 512

In [5]:
df = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/livedoor_text.csv")
print(df.shape)
df.head()

(7367, 2)


,text,category
0,27日に生放送された日本テレビ「バンクーバー2010」には、女子フィギュアスケートで銀メダル...,7
1,「腐女子」という言葉をご存知でしょうか。\nいわゆる漫画やアニメキャラなどの男性同士の恋愛（...,0
2,展示会イベント恒例のおねいさん写真のコーナーでございます \n\n国内最大級の携帯電話や無線...,6
3,芸能界を引退した島田紳助さんが、今月２８日に公開される映画「犬の首輪とコロッケと」に声だけ出...,2
4,お花に包まれた洋館で、イケメン執事に囲まれながら、ゆったりと過ごす午後のひととき……。女の子...,5


# データセットの作成

In [6]:
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.y)

  def encode(self, tokenizer, text):
      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    label = self.y[index]
    ids = []
    mask = []
    inputs = self.encode(tokenizer=self.tokenizer, text=text)
    ids.append(torch.LongTensor(inputs['input_ids']))
    mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      'label': label,
      'text':text,
      # 'userID':userID
    }

In [7]:
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

In [8]:
X = df["text"].values
y = df["category"].values

In [9]:
X_train_eval, X_test, y_train_eval, y_test = train_test_split(X, y, train_size=0.8)

X_train, X_eval, y_train, y_eval = train_test_split(X_train_eval, y_train_eval, train_size=0.75)

print(len(X_train))
print(len(X_eval))
print(len(X_test))

print(len(y_train))
print(len(y_eval))
print(len(y_test))

4419
1474
1474
4419
1474
1474


In [10]:
dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len=max_len)
dataset_eval = CreateDataset(X_eval, y_eval, tokenizer, max_len=max_len)
dataset_test = CreateDataset(X_test, y_test, tokenizer, max_len=max_len)

print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

4419
1474
1474


In [11]:
# dataset_train[0]

# データローダの作成

In [12]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, pin_memory=True)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dataloader_train, "val": dataloader_eval}

In [13]:
tmp = next(iter(dataloader_train))
print(tmp["ids"][0].size())
print(tmp["label"])
tmp["ids"][0][0]


torch.Size([16, 512])
tensor([0, 5, 6, 2, 5, 0, 7, 4, 7, 6, 6, 0, 8, 0, 3, 7])


tensor([    2, 25099,  6309,   315,     7, 10219,    47, 28504, 28468,    10,
         7946,    23,   594,    24,    28,     6,  6739,    12,     9, 13215,
          591, 28477,    18,     5,    29,     6, 12008,  7255, 30144,  7255,
            5,  8903, 28689,  4427,   199,     8,   205, 31958,     6,  9039,
        28462, 28446, 20996, 22752,    10,  5602,     6,    13, 11213, 28457,
           16,     9, 20856,     8,  7033,   737,  4354,    11, 15889,    45,
          250,  3393,     5,  1709,     9,     6,  5523,   981,     7,  6602,
         2722,   120,    75,    40,     8,   381, 16131,   250, 16019,     5,
         1709,     8,    70,   969,  2032,     5,  1208,     5,  1570,     5,
        14919,    11,   426,    15,    16,    33,     5,    14,     6, 19240,
         2334,     5,   469,  4616,     8,    36,   469,  4616,  1410,  1377,
         2397,    38,     9,  3437,    19,     5,  2991,  1601,     6,   171,
          126,    57,   429,    53,     5, 14284,  4771,    11, 

# BERTモデル

In [14]:
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", output_attentions=True, output_hidden_states=True)

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [15]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

        # カウント
        self.count = 0


    def forward(self, input_ids, attention_show_flg:bool):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        self.count += 1

        if attention_show_flg:
          return output, result.attentions[-1]
        else:
          return output


In [16]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [17]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True

In [18]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 学習・検証

In [19]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch["ids"][0].to(device)  # 文章
                labels = batch["label"].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs, attention_show_flg=False)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [20]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 3
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.3880 || 10iter. || 本イテレーションの正解率：0.1875
イテレーション 20 || Loss: 1.7851 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 30 || Loss: 1.7566 || 10iter. || 本イテレーションの正解率：0.375
イテレーション 40 || Loss: 1.5637 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 50 || Loss: 0.7577 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 60 || Loss: 1.0643 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 70 || Loss: 1.0008 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 80 || Loss: 1.2287 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 90 || Loss: 0.7291 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 100 || Loss: 0.4459 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 110 || Loss: 1.0650 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 120 || Loss: 0.3648 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 130 || Loss: 0.6504 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 140 || Loss: 0.3478 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 150 || Loss: 0.6508 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 160 || Loss: 0.6419 || 10iter. || 本イテレーションの正解率：0.812

In [ ]:
# モデル保存
# CPU版
save_path = "./drive/MyDrive/Colab_Notebooks/model/single_bert_fine_tuning_livedoor_cpu.pth"
torch.save(net_trained.to('cpu').state_dict(), save_path)

# GPU版
save_path = "./drive/MyDrive/Colab_Notebooks/model/single_bert_fine_tuning_livedoor_gpu.pth"
torch.save(net_trained.state_dict(), save_path)

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dataloader_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    inputs = batch["ids"][0].to(device)  # 文章
    labels = batch["label"].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs, attention_show_flg=False)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dataloader_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dataloader_test.dataset), epoch_acc))

100%|██████████| 93/93 [00:25<00:00,  3.63it/s]

テストデータ1474個での正解率：0.9227


# Limeによる解釈

In [78]:
! pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 275 kB 5.0 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=29fa0509dcd6e721a095a2b32244b64da1c47e89855cd71bdac8d7fb827deca2
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [79]:
import torch.nn.functional as F
import lime
from lime.lime_text import LimeTextExplainer

In [82]:
class_names = [
    'dokujo-tsushin',
    'it-life-hack',
    'smax',
    'sports-watch',
    'kaden-channel',
    'movie-enter',
    'topic-news',
    'livedoor-homme',
    'peachy'
    ]

In [101]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def predictor(texts):
    encoding = tokenizer.batch_encode_plus(texts, padding="max_length", max_length=512)
    print(encoding)
    
    input_ids = torch.tensor(encoding['input_ids']).to(device)
    print(input_ids.size())

    with torch.no_grad():
        output = net_trained(input_ids, attention_show_flg=False)
    
    probas = F.softmax(output).cpu().detach().numpy()

    return probas

In [104]:
i = 2
texts = []
texts.append(dataset_test[i]["text"])
output = predictor(texts)
output

{'input_ids': [[2, 197, 32, 4473, 333, 6, 6098, 36, 49, 1148, 14027, 301, 143, 184, 14882, 7, 9, 6, 22309, 28838, 23612, 724, 28472, 4328, 2554, 29986, 13, 6, 601, 28522, 2233, 28472, 11071, 493, 28500, 14, 128, 793, 8, 91, 542, 1213, 14, 486, 3453, 7, 7143, 11, 10570, 267, 36, 49, 1148, 14027, 18361, 38, 5, 3001, 12, 9, 6, 744, 28737, 4668, 29634, 6, 8629, 28551, 5283, 60, 5, 7143, 7, 5764, 10, 8, 22237, 6, 744, 28737, 40, 36, 7946, 75, 13, 4479, 16, 21, 2610, 29, 2935, 28471, 101, 573, 251, 18, 5602, 69, 8096, 5, 124, 7, 6373, 16, 2501, 40, 1108, 140, 7143, 14, 6202, 84, 10, 11071, 8, 36, 26590, 45, 3083, 16, 2610, 11218, 6, 23218, 28469, 20, 16, 14930, 28457, 18607, 28454, 1778, 4046, 28484, 54, 38, 13, 10880, 34, 13, 6, 59, 2524, 7, 9, 36, 23, 744, 28737, 5, 24, 1925, 212, 16, 9533, 3318, 9763, 1058, 2992, 11218, 6, 20654, 20125, 16, 2610, 54, 1852, 38, 13, 3059, 1549, 244, 28927, 8, 171, 7, 9, 6, 486, 7054, 5, 26159, 5762, 29494, 28, 36, 16329, 28523, 49, 8, 1, 7134, 630, 38, 13, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


array([[4.6288947e-06, 9.5980095e-06, 1.9094357e-06, 3.4744698e-05,
        1.3533072e-04, 2.8226789e-06, 3.3784329e-06, 9.7459829e-01,
        2.5209313e-02]], dtype=float32)

In [105]:
output.sum()

1.0000001

In [107]:
print(np.argmax(output))
print(dataset_test[i]["text"])
print(dataset_test[i]["label"])

7
12日深夜放送、テレビ朝日「やべっちF.C.」には、VVVフェンロー・吉田麻也と、フィテッセ・安田理大が生出演。日本代表メンバーが番組ゲストに質問をぶつける「やべっちQB」のコーナーでは、今野泰幸、長谷部誠らの質問に答えた。

まずは、今野から「先輩だと思っていますか？（6歳下なのに同学年のように接してくるから）」という質問が寄せられた安田。「こういうこと言ってますけど、いじられて凄い嬉しそうっすよ」と反論すると、その直後には「（今野の）写真見てもらったら分かるんですけど、エラはってますよね」と言いたい放題。これには、番組MCの矢部浩之も「ホンマや。舐めてるわ」と呆れた様子で呟いた。

また、ドイツではシャルケ・内田篤人と一緒に過ごす機会が多く、自身のブログでも度々内田の写真をアップしている吉田に対し、日本代表キャプテン・長谷部から「ブログに内田の写真を載せているようですが、他人任せでいいんですか？」というツッコミのような質問が……。

これには吉田も「他人任せじゃないですよ」と一旦は言い返したが、すぐさま「正直、アクセス数が増えるんで。やらしい話」と本音を漏らし、「（仲が良過ぎて怪しい？）普通ですよ。ないない」と、仲良いがゆえに周囲から寄せられる“疑惑”を苦笑いで否定、安田が「常に二人で部屋とかで一緒にいますからね」と、火に油を注ぐや、「なんで生でそんなこと言うんですか？」と声を荒げる吉田だった。
7


In [108]:
explainer = LimeTextExplainer(class_names=class_names)

str_to_predict = texts[0]
exp = explainer.explain_instance(str_to_predict, predictor, num_features=20, num_samples=2000)
exp.show_in_notebook(text=str_to_predict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



RuntimeError: ignored

# Attentionの可視化

In [ ]:
# BertForIMDbで処理

# ミニバッチの用意
batch = next(iter(dataloader_test))

# GPUが使えるならGPUにデータを送る
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs = batch["ids"][0].to(device)  # 文章
labels = batch["label"].to(device)  # ラベル

outputs, attention_probs = net_trained(inputs, attention_show_flg=True)

_, preds = torch.max(outputs, 1)  # ラベルを予測


In [ ]:
attention_probs.size()

torch.Size([16, 12, 512, 512])

In [ ]:
id2label = {
    0: 'dokujo-tsushin', 
    1: 'it-life-hack', 
    2: 'smax', 
    3: 'sports-watch', 
    4: 'kaden-channel', 
    5: 'movie-enter', 
    6: 'topic-news', 
    7: 'livedoor-homme', 
    8: 'peachy'
}

In [ ]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    '''
    Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数
    '''

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights):
    '''
    HTMLデータを作成する
    '''

    # indexの結果を抽出
    sentence = batch["ids"][0][index]  # 文章
    label = batch["label"][index]  # ラベル
    pred = preds[index]  # 予測

    # ラベルと予測結果を文字に置き換え
    label_str = id2label[label.item()]
    pred_str = id2label[pred.item()]

    # 表示用のHTMLを作成する
    html = f"正解ラベル：{label_str}<br>推論ラベル：{pred_str}<br><br>"

    # Self-Attentionの重みを可視化。Multi-Headが12個なので、12種類のアテンションが存在
    for i in range(12):

        # indexのAttentionを抽出と規格化
        # 0単語目[CLS]の、i番目のMulti-Head Attentionを取り出す
        # indexはミニバッチの何個目のデータかをしめす
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERTのAttentionを可視化_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 単語が[SEP]の場合は文章が終わりなのでbreak
            if tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
            html += highlight(tokenizer.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    # 12種類のAttentionの平均を求める。最大値で規格化
    all_attens = attens*0  # all_attensという変数を作成する
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERTのAttentionを可視化_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 単語が[SEP]の場合は文章が終わりなのでbreak
        if tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
        html += highlight(tokenizer.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html


In [ ]:
from IPython.display import HTML

index = 2  # 出力させたいデータ
html_output = mk_html(index, batch, preds, attention_probs)  # HTML作成
HTML(html_output)  # HTML形式で出力
